## 다운 받은 한식진흥원데이터 json 형식으로 변환
- 코드, 메뉴명, 재료및분량, 이미지(url), 조리법(제목 - 준비하기, 조리하기, 조언)
- 재료및분량 : 재료,분량 - 재료와 양념 모두 기재
- 재료 : 메인재료 - 메인, 재료들, 서브재료 - 서브명, 재료들
- 조리법 : 제목, 설명 리스트

In [186]:
from bs4 import BeautifulSoup
import os
import re
import json
def read_file(food_file):
    # print(f'file : {food_file}')
    with open(f'./foods/{food_file}') as f:
        html = f.read()
    return html

## 재료 한 라인에서 재료 정보 추출
def get_ingredients(ingredient_li):
    ul = ingredient_li.select('ul')
    title = '메인'
    ingredient_list = None
    if len(ul) > 0:
        li = ul[0].select_one('li')
        title = ingredient_li.text.split('\n')[0]
        ingredient_list = li.text.split(',')
    else:
        ingredient_list = ingredient_li.text.split(',')
    return {
        '제목' : title,
        '재료' : ingredient_list
    }

## 레스피 한 라인정보에서 레스피 정보 추출
def get_recipe(recipe):
    title = recipe.text.strip()
    sub_recipe = recipe.find_next_sibling()
    lis = sub_recipe.select('li')
    recipe_list = None
    if len(lis) > 0:
        recipe_list = [li.text.strip() for li in lis]
    else:
        recipe_list = sub_recipe.text.strip()
    return {
        '재목': title,
        '설명': recipe_list
    }
    
## 이미지 제목 요소의 이웃 요소에서 이미지 url 가져오기
def get_image_url(image):
    backgrond_url = image.find_next_sibling()['style']
    image_url = re.findall(r"url\(['\"]?([^'\"\)]{0,})['\"]?\)", backgrond_url)
    url = ''
    if len(image_url) > 0:
        url = 'https://www.hansik.or.kr/' + image_url[0]
    return url

In [176]:

## 파일 읽어 내용 추출하여 json 포맷 생성
def make_json(file):
    html = read_file(file)
    soup = BeautifulSoup(html)
    result = {}
    food_name = soup.select_one('h3').text.strip()
    food_detail = soup.select_one('.col-lg-4')
    result['코드'] = id
    result['메뉴명'] = food_name
    ## 재료및 분량
    food_qty = soup.select_one('.col-lg-4 h4 span').text[1:-1]
    food_ingredients = [get_ingredients(ingredient) for ingredient in soup.select('.list-ingredients > li')]
    result['분량'] = food_qty
    result['재료'] = food_ingredients
    image = soup.select_one('.col-lg-8 h4')
    recipes = soup.select('.col-lg-8 h4')[-3:]
    image_url = get_image_url(image)
    recipe_list = [get_recipe(recipe) for recipe in recipes]
    result['이미지'] = image_url
    result['조리법'] = recipe_list
    print(result)
    return result

In [164]:
file_list = os.listdir('./foods')
f'./foods/{file_list[0]}'

'./foods/56756_흰밥.text'

In [187]:
for file in os.listdir('./foods'):
    food_json = make_json(file)
    print(f"./json/{file.split('.')[0]}.json")
    with open(f"./json/{file.split('.')[0]}.json", "w", encoding="utf-8") as f:
        json.dump(food_json, f, indent=4, ensure_ascii=False)

{'코드': '56756', '메뉴명': '흰밥', '분량': '4인분', '재료': [{'제목': '메인', '재료': ['멥쌀 2컵(450g)', ' 물 2½컵(625ml)']}], '이미지': 'https://www.hansik.or.kr//resources/img/recipe/food001.jpg', '조리법': [{'재목': '준비하기', '설명': ['쌀은 밥을 짓기 30분 전에 깨끗이 씻어 물에 불린 뒤 체에 건져 물기를 뺀다. 이렇게 해야 밥이 질어지지 않고 잘 익는다.']}, {'재목': '조리하기', '설명': ['냄비에 씻어 불린 쌀을 담고 물 2½컵을 부은 다음 뚜껑을 덮고 센 불에 올려 끓인다.', '한 번 끓어오르면 약한 불로 줄여서 20분간 끓인다. 쌀알이 퍼지면 불을 아주 약하게 줄이고 5분 동안 뜸을 들인다.', '주걱으로 위아래를 살살 섞은 뒤 그릇에 담아낸다.']}, {'재목': '조언', '설명': '멥쌀은 도정 정도에 따라 현미, 5분 도미, 7 분 도미, 백미로 분류한다. 육안으로 보았을 때 멥쌀은 반투명하고 찹쌀은 유백색이다.'}]}
./json/56756_흰밥.json
{'코드': '56756', '메뉴명': '오곡밥', '분량': '4인분', '재료': [{'제목': '메인', '재료': ['멥쌀 2컵(450g)', ' 찹쌀 1컵(220g)', ' 팥 ¼컵', ' 검은콩 ½컵', ' 수수 ½컵', ' 차조 ¼컵']}, {'제목': '밥물', '재료': ['2½컵(625ml', ' 팥 삶은 물 1컵', ' 물 1½컵)', ' 소금 ½큰술']}], '이미지': 'https://www.hansik.or.kr//resources/img/recipe/food002.jpg', '조리법': [{'재목': '준비하기', '설명': ['멥쌀과 찹쌀은 밥 짓기 30분 전에 씻어 물에 불렸다가 건진다.', '팥은 깨끗이 씻어 냄비에 담고 충분히 잠길 정도로 물 2컵을 부어 불에 올린다. 끓어오르면 물은 따라 버리고, 다시 3컵 정도의 

In [181]:
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(food_json, f, indent=4, ensure_ascii=False)